In [596]:
from matplotlib.colors import LogNorm
import numpy as np
import pandas as pd
import seaborn as sns
import os
import glob
from datetime import datetime
from datetime import timedelta
from matplotlib import pyplot as plt
import matplotlib.dates as md
import matplotlib.colors as mcolors
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import warnings
from mpl_toolkits.basemap import Basemap
from scipy.interpolate import interp2d
warnings.filterwarnings('ignore')
#import datetime
import scipy.ndimage as ndimage
from matplotlib import cm
import geopy.distance
#import matplotlib as mpl

from shapely.geometry import Point
import geopandas as gpd
from geopandas import GeoDataFrame
import leafmap
import plotly.express as px
from matplotlib.transforms import ScaledTranslation
from matplotlib.collections import LineCollection
from matplotlib.colors import ListedColormap, BoundaryNorm
#import pysplit

In [597]:
def simple_read(files):
    # Initialize an empty DataFrame to store the data from all files
    all_data = []

    for path in files:
        with open(path) as f:
            first_line = f.readline()
            header_line = int(first_line[0:-2].split(",")[0]) - 1

        data = pd.read_csv(path, sep=',', skiprows=header_line)
        data.columns = [i.strip() for i in data.columns]

        acc = 0
        boo = False
        for letter in path:
            if letter == '2':
                boo = True
            elif boo and letter == '0':
                acc -= 1
                break
            acc += 1

        dt = datetime(int(path[acc:acc+4]), int(path[acc+4:acc+6]), int(path[acc+6:acc+8]))

        for column in data.columns:
            if 'Time' in column:
                data[column] = dt + pd.to_timedelta(data[column], unit='seconds')

        all_data.append(data.replace(-999999, np.NaN))

    # Concatenate all DataFrames in the list to combine data from all files
    combined_data = pd.concat(all_data, ignore_index=True)
    
    return combined_data


In [598]:
file_dir = 'C:/Users/taiwoajayi/Documents/pil/'

files_2022 = np.sort(glob.glob(file_dir + '**activate*2022**.ict'))

ds_2022 = simple_read(files_2022)
ds_2022 = ds_2022.replace(-888888, 0)
ds_2022 = ds_2022.dropna()

# Reset the index after removing rows
ds_2022.reset_index(drop=True, inplace=True)
# Filter the data using the condition >= 0.4
#ds_2022 = ds_2022[(ds_2022['Sodium_ZIEMBA'] >= 0.1) & (ds_2022['Sodium_ZIEMBA'] <= 2)]
ds_2022 = ds_2022.dropna()
ds_2022

,Time_Start,Time_Stop,Latitude_THORNHILL,Longitude_THORNHILL,GPS_Altitude_THORNHILL,Sodium_ZIEMBA,Ammonium_ZIEMBA,Potassium_ZIEMBA,Magnesium_ZIEMBA,Calcium_ZIEMBA,Chloride_ZIEMBA,Nitrate_ZIEMBA,Sulfate_ZIEMBA,Oxalate_ZIEMBA
0,2022-06-02 11:19:14,2022-06-02 11:19:15,32.363607,-64.676112,-27.670,0.9589,4.293,0.0,0.0454,0.008656,0.9537,0.9028,1.626,0.00916
1,2022-06-02 11:19:15,2022-06-02 11:19:16,32.363719,-64.676758,-27.460,0.9589,4.293,0.0,0.0454,0.008656,0.9537,0.9028,1.626,0.00916
2,2022-06-02 11:19:16,2022-06-02 11:19:17,32.363850,-64.677509,-27.115,0.9589,4.293,0.0,0.0454,0.008656,0.9537,0.9028,1.626,0.00916
3,2022-06-02 11:19:17,2022-06-02 11:19:18,32.363985,-64.678276,-26.515,0.9589,4.293,0.0,0.0454,0.008656,0.9537,0.9028,1.626,0.00916
4,2022-06-02 11:19:18,2022-06-02 11:19:19,32.364120,-64.679059,-25.485,0.9589,4.293,0.0,0.0454,0.008656,0.9537,0.9028,1.626,0.00916
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155230,2022-06-17 16:56:00,2022-06-17 16:56:01,32.372309,-64.726388,172.380,1.2820,8.013,0.0,0.0341,0.000000,3.3660,0.3732,0.689,0.01352
155231,2022-06-17 16:56:01,2022-06-17 16:56:02,32.372208,-64.725767,170.085,1.2820,8.013,0.0,0.0341,0.000000,3.3660,0.3732,0.689,0.01352
155232,2022-06-17 16:56:02,2022-06-17 16:56:03,32.372103,-64.725148,167.540,1.2820,8.013,0.0,0.0341,0.000000,3.3660,0.3732,0.689,0.01352
155233,2022-06-17 16:56:03,2022-06-17 16:56:04,32.371995,-64.724529,165.040,1.2820,8.013,0.0,0.0341,0.000000,3.3660,0.3732,0.689,0.01352


In [599]:
file_dir = 'C:/Users/taiwoajayi/Documents/pills/'

files_2020 = np.sort(glob.glob(file_dir + '**ACTIVATE*2022**.ict'))

ds_2020 = simple_read(files_2020)
ds_2020 = ds_2020.replace(-8888, np.NaN)
ds_2020 = ds_2020.replace(-9999, np.NaN)
ds_2020 = ds_2020.dropna()
#ds_2020 = ds_2020[ds_2020['Chloride'] <= 2]
#ds_2020 = ds_2020[ds_2020['Sodium'] <= 4 ]
ds_2020

,Time_Start,Time_Stop,Time_Mid,Sodium,Ammonium,Potassium,Magnesium,Calcium,Chloride,Nitrate,Sulfate,Oxalate
5,2022-06-02 11:44:31,2022-06-02 11:49:31,2022-06-02 11:47:01,1.7060,3.761,0.04663,0.14650,0.096400,2.0080,1.1770,2.2490,0.005919
10,2022-06-02 12:10:06,2022-06-02 12:15:06,2022-06-02 12:12:36,0.8202,2.750,0.02384,0.14460,0.089450,0.9183,0.8811,1.7120,0.008450
11,2022-06-02 12:15:13,2022-06-02 12:20:13,2022-06-02 12:17:43,1.7170,5.890,0.13420,0.23950,0.203300,1.4200,1.5630,2.5260,0.010180
13,2022-06-02 12:25:27,2022-06-02 12:30:27,2022-06-02 12:27:57,1.5250,4.616,0.05147,0.12590,0.125900,1.5160,1.8910,2.2200,0.006162
15,2022-06-02 12:35:41,2022-06-02 12:40:41,2022-06-02 12:38:11,0.9632,4.691,0.00981,0.04487,0.020680,1.1930,1.2160,1.6260,0.004473
...,...,...,...,...,...,...,...,...,...,...,...,...
523,2022-06-17 15:55:01,2022-06-17 16:00:01,2022-06-17 15:57:31,2.9810,8.151,0.11310,0.25440,0.009252,4.2200,0.3282,0.9037,0.012350
526,2022-06-17 16:10:22,2022-06-17 16:15:22,2022-06-17 16:12:52,5.6030,11.170,0.26140,0.20310,0.002252,8.0050,0.7198,1.5430,0.016130
527,2022-06-17 16:15:29,2022-06-17 16:20:29,2022-06-17 16:17:59,4.9670,3.470,0.57970,0.34730,0.097950,8.7310,1.1590,0.8639,0.032710
528,2022-06-17 16:20:36,2022-06-17 16:25:36,2022-06-17 16:23:06,4.3930,7.169,0.20290,0.39900,0.041780,7.1020,1.5250,1.7200,0.011450


In [600]:
'''start_time = pd.to_datetime("2022-06-02 13:48:00")
stop_time = pd.to_datetime("2022-06-02 14:10:00")

# Find the nearest indices to the target timestamps
start_index = np.argmin(np.abs(ds_2022["Time_Start"] - start_time))
stop_index = np.argmin(np.abs(ds_2022["Time_Start"] - stop_time))

# Extract the subset of data
PILLS_2_subset = ds_2022.iloc[start_index:stop_index + 1]  # Add 1 to include the stop_index
PILLS_2_subset.mean()'''


'start_time = pd.to_datetime("2022-06-02 13:48:00")\nstop_time = pd.to_datetime("2022-06-02 14:10:00")\n\n# Find the nearest indices to the target timestamps\nstart_index = np.argmin(np.abs(ds_2022["Time_Start"] - start_time))\nstop_index = np.argmin(np.abs(ds_2022["Time_Start"] - stop_time))\n\n# Extract the subset of data\nPILLS_2_subset = ds_2022.iloc[start_index:stop_index + 1]  # Add 1 to include the stop_index\nPILLS_2_subset.mean()'

In [662]:
# Specify the target timestamps
start_time = pd.to_datetime("2022-06-02 13:48:00")
stop_time = pd.to_datetime("2022-06-02 14:10:00")

# Find the nearest indices to the target timestamps
start_index = np.argmin(np.abs(ds_2020["Time_Start"] - start_time))
stop_index = np.argmin(np.abs(ds_2020["Time_Start"] - stop_time))

# Extract the subset of data
PILS_2_subset = ds_2020.iloc[start_index:stop_index + 1]  # Add 1 to include the stop_index
PILS_2_subset2 = PILS_2_subset.drop('Ammonium', axis=1)
PILS_2_subset2.mean()



Time_Start    2022-06-02 11:44:31
Time_Stop     2022-06-02 11:49:31
Time_Mid      2022-06-02 11:47:01
Sodium                      1.706
Potassium                 0.04663
Magnesium                  0.1465
Calcium                    0.0964
Chloride                    2.008
Nitrate                     1.177
Sulfate                     2.249
Oxalate                  0.005919
dtype: object

In [602]:
# Specify the target timestamps
start_time = pd.to_datetime("2022-06-02 13:48:00")
stop_time = pd.to_datetime("2022-06-02 14:10:00")

# Find the nearest indices to the target timestamps
start_index = np.argmin(np.abs(ds_2020["Time_Start"] - start_time))
stop_index = np.argmin(np.abs(ds_2020["Time_Start"] - stop_time))

# Extract the subset of data
PILS_2_subset = ds_2020.iloc[start_index:stop_index + 1]  # Add 1 to include the stop_index
PILS_2_subset2 = PILS_2_subset.drop('Ammonium', axis=1)

# Select the columns you want to sum
columns_to_sum = ['Sodium', 'Potassium', 'Magnesium', 'Calcium', 'Chloride', 'Nitrate', 'Sulfate', 'Oxalate']

# Sum the selected columns and store the result in Africa_total1
PILS_2_subset1 = PILS_2_subset2[columns_to_sum].sum(axis = 1)
# Calculate the sum of values for each row (axis=1)

# Calculate the sum of all the values in Africa_total1
#Africa_total1 = Africa_total1.sum()

# Specify the range of columns for division
start_col2 = 'Sodium'
stop_col2 = 'Oxalate'

# Perform division and assign the result to the specified range of columns
PILS_2_subset2.loc[:, start_col2:stop_col2] = PILS_2_subset2.loc[:, start_col2:stop_col2].div(PILS_2_subset1, axis=0)
PILS_2_subset2['Nso4'] = PILS_2_subset2['Sulfate'] - (0.253 * PILS_2_subset2['Sodium'])
PILS_2_subset2['ratio'] = PILS_2_subset2['Chloride'] / PILS_2_subset2['Sodium']
PILS_2_subset2.mean()


Time_Start    2022-06-02 13:57:33.000000256
Time_Stop     2022-06-02 14:02:33.000000256
Time_Mid      2022-06-02 14:00:02.999999744
Sodium                             0.212202
Potassium                           0.01308
Magnesium                          0.017344
Calcium                            0.023762
Chloride                           0.190217
Nitrate                            0.215283
Sulfate                            0.326307
Oxalate                            0.001806
Nso4                                0.27262
ratio                              0.932139
dtype: object

In [603]:
PILS_2_subset1.mean()

5.240473

In [604]:
# Specify the target timestamps
start_time = pd.to_datetime("2022-06-02 18:36:00")
stop_time = pd.to_datetime("2022-06-02 18:53:00")

# Find the nearest indices to the target timestamps
start_index = np.argmin(np.abs(ds_2020["Time_Start"] - start_time))
stop_index = np.argmin(np.abs(ds_2020["Time_Start"] - stop_time))

# Extract the subset of data
PILS_2b_subset = ds_2020.iloc[start_index:stop_index + 1]  # Add 1 to include the stop_index
# Extract the subset of data
PILS_2b_subset2 = PILS_2b_subset.drop('Ammonium', axis=1)

# Select the columns you want to sum
columns_to_sum = ['Sodium', 'Potassium', 'Magnesium', 'Calcium', 'Chloride', 'Nitrate', 'Sulfate', 'Oxalate']

# Sum the selected columns and store the result in Africa_total1
PILS_2b_subset1 = PILS_2b_subset2[columns_to_sum].sum(axis = 1)
# Calculate the sum of values for each row (axis=1)

# Calculate the sum of all the values in Africa_total1
#Africa_total1 = Africa_total1.sum()

# Specify the range of columns for division
start_col2 = 'Sodium'
stop_col2 = 'Oxalate'

# Perform division and assign the result to the specified range of columns
PILS_2b_subset2.loc[:, start_col2:stop_col2] = PILS_2b_subset2.loc[:, start_col2:stop_col2].div(PILS_2b_subset1, axis=0)
PILS_2b_subset2.mean()

Time_Start    2022-06-02 18:38:05.500000
Time_Stop     2022-06-02 18:43:05.500000
Time_Mid      2022-06-02 18:40:35.500000
Sodium                          0.399047
Potassium                        0.02283
Magnesium                       0.039483
Calcium                         0.025075
Chloride                        0.059761
Nitrate                         0.161643
Sulfate                         0.284086
Oxalate                         0.008074
dtype: object

In [605]:
PILS_2b_subset2['Nso4'] = PILS_2b_subset2['Sulfate'] - (0.253 * PILS_2b_subset2['Sodium'])
PILS_2b_subset2['ratio'] = PILS_2b_subset2['Chloride'] / PILS_2b_subset2['Sodium']
PILS_2b_subset2.mean()

Time_Start    2022-06-02 18:38:05.500000
Time_Stop     2022-06-02 18:43:05.500000
Time_Mid      2022-06-02 18:40:35.500000
Sodium                          0.399047
Potassium                        0.02283
Magnesium                       0.039483
Calcium                         0.025075
Chloride                        0.059761
Nitrate                         0.161643
Sulfate                         0.284086
Oxalate                         0.008074
Nso4                            0.183127
ratio                           0.269968
dtype: object

In [606]:
PILS_2b_subset1.mean()

2.0903645

In [607]:
# Specify the target timestamps
start_time = pd.to_datetime("2022-06-02 12:59:00")
stop_time = pd.to_datetime("2022-06-02 13:10:00")

# Find the nearest indices to the target timestamps
start_index = np.argmin(np.abs(ds_2020["Time_Start"] - start_time))
stop_index = np.argmin(np.abs(ds_2020["Time_Start"] - stop_time))

# Extract the subset of data
PILS_22_subset = ds_2020.iloc[start_index:stop_index + 1]  # Add 1 to include the stop_index
PILS_22_subset2 = PILS_22_subset.drop('Ammonium', axis=1)

# Select the columns you want to sum
columns_to_sum = ['Sodium', 'Potassium', 'Magnesium', 'Calcium', 'Chloride', 'Nitrate', 'Sulfate', 'Oxalate']

# Sum the selected columns and store the result in Africa_total1
PILS_22_subset1 = PILS_22_subset2[columns_to_sum].sum(axis = 1)
# Calculate the sum of values for each row (axis=1)

# Calculate the sum of all the values in Africa_total1
#Africa_total1 = Africa_total1.sum()

# Specify the range of columns for division
start_col2 = 'Sodium'
stop_col2 = 'Oxalate'

# Perform division and assign the result to the specified range of columns
PILS_22_subset2.loc[:, start_col2:stop_col2] = PILS_22_subset2.loc[:, start_col2:stop_col2].div(PILS_22_subset1, axis=0)
PILS_22_subset2.mean()

Time_Start    2022-06-02 13:01:16
Time_Stop     2022-06-02 13:06:16
Time_Mid      2022-06-02 13:03:46
Sodium                     0.1658
Potassium                0.010552
Magnesium                0.030473
Calcium                  0.035747
Chloride                 0.147273
Nitrate                  0.255792
Sulfate                  0.349479
Oxalate                  0.004883
dtype: object

In [608]:
PILS_22_subset2['Nso4'] = PILS_22_subset2['Sulfate'] - (0.253 * PILS_22_subset2['Sodium'])
PILS_22_subset2['ratio'] = PILS_22_subset2['Chloride'] / PILS_22_subset2['Sodium']
PILS_22_subset2.mean()

Time_Start    2022-06-02 13:01:16
Time_Stop     2022-06-02 13:06:16
Time_Mid      2022-06-02 13:03:46
Sodium                     0.1658
Potassium                0.010552
Magnesium                0.030473
Calcium                  0.035747
Chloride                 0.147273
Nitrate                  0.255792
Sulfate                  0.349479
Oxalate                  0.004883
Nso4                     0.307532
ratio                    0.888253
dtype: object

In [609]:
PILS_22_subset1.mean()

2.002408

In [610]:
# Specify the target timestamps
start_time = pd.to_datetime("2022-06-07 14:32:00")
stop_time = pd.to_datetime("2022-06-07 14:48:00")

# Find the nearest indices to the target timestamps
start_index = np.argmin(np.abs(ds_2020["Time_Start"] - start_time))
stop_index = np.argmin(np.abs(ds_2020["Time_Start"] - stop_time))

# Extract the subset of data
PILS_7_subset = ds_2020.iloc[start_index:stop_index + 1]  # Add 1 to include the stop_index
PILS_7_subset2 = PILS_7_subset.drop('Ammonium', axis=1)
columns_to_sum = ['Sodium', 'Potassium', 'Magnesium', 'Calcium', 'Chloride', 'Nitrate', 'Sulfate', 'Oxalate']

# Sum the selected columns and store the result in Africa_total1
PILS_7_subset1 = PILS_7_subset2[columns_to_sum].sum(axis = 1)
# Calculate the sum of values for each row (axis=1)

# Calculate the sum of all the values in Africa_total1
#Africa_total1 = Africa_total1.sum()

# Specify the range of columns for division
start_col2 = 'Sodium'
stop_col2 = 'Oxalate'

# Perform division and assign the result to the specified range of columns
PILS_7_subset2.loc[:, start_col2:stop_col2] = PILS_7_subset2.loc[:, start_col2:stop_col2].div(PILS_7_subset1, axis=0)
PILS_7_subset2.mean()

Time_Start    2022-06-07 14:33:35.500000
Time_Stop     2022-06-07 14:38:35.500000
Time_Mid      2022-06-07 14:36:05.500000
Sodium                          0.292861
Potassium                       0.006478
Magnesium                       0.023346
Calcium                         0.012205
Chloride                        0.453495
Nitrate                         0.096019
Sulfate                          0.11435
Oxalate                         0.001246
dtype: object

In [611]:
PILS_7_subset2['Nso4'] = PILS_7_subset2['Sulfate'] - (0.253 * PILS_7_subset2['Sodium'])
PILS_7_subset2['ratio'] = PILS_7_subset2['Chloride'] / PILS_7_subset2['Sodium']
PILS_7_subset2.mean()

Time_Start    2022-06-07 14:33:35.500000
Time_Stop     2022-06-07 14:38:35.500000
Time_Mid      2022-06-07 14:36:05.500000
Sodium                          0.292861
Potassium                       0.006478
Magnesium                       0.023346
Calcium                         0.012205
Chloride                        0.453495
Nitrate                         0.096019
Sulfate                          0.11435
Oxalate                         0.001246
Nso4                            0.040256
ratio                           1.549955
dtype: object

In [612]:
PILS_7_subset1.mean()

20.028405

In [613]:
# Specify the target timestamps
start_time = pd.to_datetime("2022-06-07 19:08:00")
stop_time = pd.to_datetime("2022-06-07 19:23:00")

# Find the nearest indices to the target timestamps
start_index = np.argmin(np.abs(ds_2020["Time_Start"] - start_time))
stop_index = np.argmin(np.abs(ds_2020["Time_Start"] - stop_time))

# Extract the subset of data
PILS_7b_subset = ds_2020.iloc[start_index:stop_index + 1]  # Add 1 to include the stop_index
PILS_7b_subset2 = PILS_7b_subset.drop('Ammonium', axis=1)
columns_to_sum = ['Sodium', 'Potassium', 'Magnesium', 'Calcium', 'Chloride', 'Nitrate', 'Sulfate', 'Oxalate']

# Sum the selected columns and store the result in Africa_total1
PILS_7b_subset1 = PILS_7b_subset2[columns_to_sum].sum(axis = 1)
# Calculate the sum of values for each row (axis=1)

# Calculate the sum of all the values in Africa_total1
#Africa_total1 = Africa_total1.sum()

# Specify the range of columns for division
start_col2 = 'Sodium'
stop_col2 = 'Oxalate'

# Perform division and assign the result to the specified range of columns
PILS_7b_subset2.loc[:, start_col2:stop_col2] = PILS_7b_subset2.loc[:, start_col2:stop_col2].div(PILS_7b_subset1, axis=0)
PILS_7b_subset2.mean()

Time_Start    2022-06-07 19:07:24
Time_Stop     2022-06-07 19:12:24
Time_Mid      2022-06-07 19:09:54
Sodium                   0.545755
Potassium                0.010564
Magnesium                0.036181
Calcium                  0.003349
Chloride                 0.323463
Nitrate                  0.028771
Sulfate                  0.049718
Oxalate                    0.0022
dtype: object

In [614]:
#PILS_7b_subset2['Nso4'] = PILS_7b_subset2['Sulfate'] - (0.253 * PILS_7b_subset2['Sodium'])
#PILS_7b_subset2['Nso4'] = PILS_7b_subset2['Sulfate'] - (0.253 * PILS_7b_subset2['Sodium'])
PILS_7b_subset2['ratio'] = PILS_7b_subset2['Chloride'] / PILS_7b_subset2['Sodium']
#PILS_7b_subset2['ratio'] [np.isinf(PILS_7b_subset2['ratio'] )] = 0
PILS_7b_subset2.mean()


Time_Start    2022-06-07 19:07:24
Time_Stop     2022-06-07 19:12:24
Time_Mid      2022-06-07 19:09:54
Sodium                   0.545755
Potassium                0.010564
Magnesium                0.036181
Calcium                  0.003349
Chloride                 0.323463
Nitrate                  0.028771
Sulfate                  0.049718
Oxalate                    0.0022
ratio                    0.592689
dtype: object

In [615]:
PILS_7b_subset1.mean()

7.7195800000000006

In [616]:
# Specify the target timestamps
start_time = pd.to_datetime("2022-06-08 20:30:00")
stop_time = pd.to_datetime("2022-06-08 20:45:00")

# Find the nearest indices to the target timestamps
start_index = np.argmin(np.abs(ds_2020["Time_Start"] - start_time))
stop_index = np.argmin(np.abs(ds_2020["Time_Start"] - stop_time))

# Extract the subset of data
PILS_8_subset = ds_2020.iloc[start_index:stop_index + 1]  # Add 1 to include the stop_index

PILS_8_subset2 = PILS_8_subset.drop('Ammonium', axis=1)
columns_to_sum = ['Sodium', 'Potassium', 'Magnesium', 'Calcium', 'Chloride', 'Nitrate', 'Sulfate', 'Oxalate']

# Sum the selected columns and store the result in Africa_total1
PILS_8_subset1 = PILS_8_subset2[columns_to_sum].sum(axis = 1)
# Calculate the sum of values for each row (axis=1)

# Calculate the sum of all the values in Africa_total1
#Africa_total1 = Africa_total1.sum()

# Specify the range of columns for division
start_col2 = 'Sodium'
stop_col2 = 'Oxalate'

# Perform division and assign the result to the specified range of columns
PILS_8_subset2.loc[:, start_col2:stop_col2] = PILS_8_subset2.loc[:, start_col2:stop_col2].div(PILS_8_subset1, axis=0)
PILS_8_subset2.mean()

Time_Start    2022-06-08 20:32:59.500000
Time_Stop     2022-06-08 20:37:59.500000
Time_Mid      2022-06-08 20:35:29.500000
Sodium                          0.321195
Potassium                       0.005278
Magnesium                       0.029024
Calcium                         0.018945
Chloride                        0.467349
Nitrate                         0.035815
Sulfate                         0.121537
Oxalate                         0.000856
dtype: object

In [617]:
PILS_8_subset2['Nso4'] = PILS_8_subset2['Sulfate'] - (0.253 * PILS_8_subset2['Sodium'])
PILS_8_subset2['ratio'] = PILS_8_subset2['Chloride'] / PILS_8_subset2['Sodium']
PILS_8_subset2.mean()

Time_Start    2022-06-08 20:32:59.500000
Time_Stop     2022-06-08 20:37:59.500000
Time_Mid      2022-06-08 20:35:29.500000
Sodium                          0.321195
Potassium                       0.005278
Magnesium                       0.029024
Calcium                         0.018945
Chloride                        0.467349
Nitrate                         0.035815
Sulfate                         0.121537
Oxalate                         0.000856
Nso4                            0.040275
ratio                           1.466748
dtype: object

In [618]:
PILS_8_subset1.mean()

9.629385

In [619]:

# Specify the target timestamps
start_time = pd.to_datetime("2022-06-10 15:17:00")
stop_time = pd.to_datetime("2022-06-10 15:25:00")

# Find the nearest indices to the target timestamps
start_index = np.argmin(np.abs(ds_2020["Time_Start"] - start_time))
stop_index = np.argmin(np.abs(ds_2020["Time_Start"] - stop_time))

# Extract the subset of data
PILS_10_subset = ds_2020.iloc[start_index:stop_index + 1]  # Add 1 to include the stop_index

PILS_10_subset2 = PILS_10_subset.drop('Ammonium', axis=1)
columns_to_sum = ['Sodium', 'Potassium', 'Magnesium', 'Calcium', 'Chloride', 'Nitrate', 'Sulfate', 'Oxalate']

# Sum the selected columns and store the result in Africa_total1
PILS_10_subset1 = PILS_10_subset2[columns_to_sum].sum(axis = 1)
# Calculate the sum of values for each row (axis=1)

# Calculate the sum of all the values in Africa_total1
#Africa_total1 = Africa_total1.sum()

# Specify the range of columns for division
start_col2 = 'Sodium'
stop_col2 = 'Oxalate'

# Perform division and assign the result to the specified range of columns
PILS_10_subset2.loc[:, start_col2:stop_col2] = PILS_10_subset2.loc[:, start_col2:stop_col2].div(PILS_10_subset1, axis=0)
PILS_10_subset2.mean()

Time_Start    2022-06-10 15:14:19
Time_Stop     2022-06-10 15:19:19
Time_Mid      2022-06-10 15:16:49
Sodium                   0.255946
Potassium                0.003973
Magnesium                0.012827
Calcium                  0.010569
Chloride                 0.417275
Nitrate                  0.093661
Sulfate                  0.205049
Oxalate                    0.0007
dtype: object

In [620]:
PILS_10_subset2['Nso4'] = PILS_10_subset2['Sulfate'] - (0.253 * PILS_10_subset2['Sodium'])
PILS_10_subset2['ratio'] = PILS_10_subset2['Chloride'] / PILS_10_subset2['Sodium']
PILS_10_subset2.mean()

Time_Start    2022-06-10 15:14:19
Time_Stop     2022-06-10 15:19:19
Time_Mid      2022-06-10 15:16:49
Sodium                   0.255946
Potassium                0.003973
Magnesium                0.012827
Calcium                  0.010569
Chloride                 0.417275
Nitrate                  0.093661
Sulfate                  0.205049
Oxalate                    0.0007
Nso4                     0.140295
ratio                    1.630322
dtype: object

In [621]:
PILS_10_subset1.mean()

7.525021

In [622]:
# Specify the target timestamps
start_time = pd.to_datetime("2022-06-10 19:30:00")
stop_time = pd.to_datetime("2022-06-10 19:51:00")

# Find the nearest indices to the target timestamps
start_index = np.argmin(np.abs(ds_2020["Time_Start"] - start_time))
stop_index = np.argmin(np.abs(ds_2020["Time_Start"] - stop_time))

# Extract the subset of data
PILS_10b_subset = ds_2020.iloc[start_index:stop_index + 1]  # Add 1 to include the stop_index
PILS_10b_subset2 = PILS_10b_subset.drop('Ammonium', axis=1)
columns_to_sum = ['Sodium', 'Potassium', 'Magnesium', 'Calcium', 'Chloride', 'Nitrate', 'Sulfate', 'Oxalate']

# Sum the selected columns and store the result in Africa_total1
PILS_10b_subset1 = PILS_10b_subset2[columns_to_sum].sum(axis = 1)
# Calculate the sum of values for each row (axis=1)

# Calculate the sum of all the values in Africa_total1
#Africa_total1 = Africa_total1.sum()

# Specify the range of columns for division
start_col2 = 'Sodium'
stop_col2 = 'Oxalate'

# Perform division and assign the result to the specified range of columns
PILS_10b_subset2.loc[:, start_col2:stop_col2] = PILS_10b_subset2.loc[:, start_col2:stop_col2].div(PILS_10b_subset1, axis=0)

# Perform division and assign the result to the specified range of columns
PILS_10b_subset2.mean()

Time_Start    2022-06-10 19:34:14.500000
Time_Stop     2022-06-10 19:39:14.500000
Time_Mid      2022-06-10 19:36:44.500000
Sodium                          0.359135
Potassium                       0.011065
Magnesium                        0.03299
Calcium                         0.030949
Chloride                        0.154497
Nitrate                         0.178926
Sulfate                         0.229993
Oxalate                         0.002445
dtype: object

In [623]:
PILS_10b_subset2['Nso4'] = PILS_10b_subset2['Sulfate'] - (0.253 * PILS_10b_subset2['Sodium'])
PILS_10b_subset2['ratio'] = PILS_10b_subset2['Chloride'] / PILS_10b_subset2['Sodium']
PILS_10b_subset2.mean()

Time_Start    2022-06-10 19:34:14.500000
Time_Stop     2022-06-10 19:39:14.500000
Time_Mid      2022-06-10 19:36:44.500000
Sodium                          0.359135
Potassium                       0.011065
Magnesium                        0.03299
Calcium                         0.030949
Chloride                        0.154497
Nitrate                         0.178926
Sulfate                         0.229993
Oxalate                         0.002445
Nso4                            0.139132
ratio                           0.606641
dtype: object

In [624]:
PILS_10b_subset1.mean()

3.6980855

In [625]:
# Specify the target timestamps
start_time = pd.to_datetime("2022-06-10 20:25:00")
stop_time = pd.to_datetime("2022-06-10 20:35:00")

# Find the nearest indices to the target timestamps
start_index = np.argmin(np.abs(ds_2020["Time_Start"] - start_time))
stop_index = np.argmin(np.abs(ds_2020["Time_Start"] - stop_time))

# Extract the subset of data
PILS_10c_subset = ds_2020.iloc[start_index:stop_index + 1]  # Add 1 to include the stop_index
PILS_10c_subset2 = PILS_10c_subset.drop('Ammonium', axis=1)
columns_to_sum = ['Sodium', 'Potassium', 'Magnesium', 'Calcium', 'Chloride', 'Nitrate', 'Sulfate', 'Oxalate']

# Sum the selected columns and store the result in Africa_total1
PILS_10c_subset1 = PILS_10c_subset2[columns_to_sum].sum(axis = 1)
# Calculate the sum of values for each row (axis=1)

# Calculate the sum of all the values in Africa_total1
#Africa_total1 = Africa_total1.sum()

# Specify the range of columns for division
start_col2 = 'Sodium'
stop_col2 = 'Oxalate'

# Perform division and assign the result to the specified range of columns
PILS_10c_subset2.loc[:, start_col2:stop_col2] = PILS_10c_subset2.loc[:, start_col2:stop_col2].div(PILS_10c_subset1, axis=0)
PILS_10c_subset2.mean()

Time_Start    2022-06-10 20:25:24.500000
Time_Stop     2022-06-10 20:30:24.500000
Time_Mid      2022-06-10 20:27:54.500000
Sodium                          0.195736
Potassium                       0.010996
Magnesium                       0.024598
Calcium                         0.010823
Chloride                        0.300629
Nitrate                         0.181603
Sulfate                           0.2689
Oxalate                         0.006715
dtype: object

In [626]:
PILS_10c_subset2['Nso4'] = PILS_10c_subset2['Sulfate'] - (0.253 * PILS_10c_subset2['Sodium'])
PILS_10c_subset2['ratio'] = PILS_10c_subset2['Chloride'] / PILS_10c_subset2['Sodium']
PILS_10c_subset2.mean()

Time_Start    2022-06-10 20:25:24.500000
Time_Stop     2022-06-10 20:30:24.500000
Time_Mid      2022-06-10 20:27:54.500000
Sodium                          0.195736
Potassium                       0.010996
Magnesium                       0.024598
Calcium                         0.010823
Chloride                        0.300629
Nitrate                         0.181603
Sulfate                           0.2689
Oxalate                         0.006715
Nso4                            0.219379
ratio                           1.517719
dtype: object

In [627]:
PILS_10c_subset1.mean()

7.467865

In [628]:
# Specify the target timestamps
start_time = pd.to_datetime("2022-06-11 15:24:00")
stop_time = pd.to_datetime("2022-06-11 15:42:00")

# Find the nearest indices to the target timestamps
start_index = np.argmin(np.abs(ds_2020["Time_Start"] - start_time))
stop_index = np.argmin(np.abs(ds_2020["Time_Start"] - stop_time))

# Extract the subset of data
PILS_11_subset = ds_2020.iloc[start_index:stop_index + 1]  # Add 1 to include the stop_index

PILS_11_subset2 = PILS_11_subset.drop('Ammonium', axis=1)
columns_to_sum = ['Sodium', 'Potassium', 'Magnesium', 'Calcium', 'Chloride', 'Nitrate', 'Sulfate', 'Oxalate']

# Sum the selected columns and store the result in Africa_total1
PILS_11_subset1 = PILS_11_subset2[columns_to_sum].sum(axis = 1)
# Calculate the sum of values for each row (axis=1)

# Calculate the sum of all the values in Africa_total1
#Africa_total1 = Africa_total1.sum()

# Specify the range of columns for division
start_col2 = 'Sodium'
stop_col2 = 'Oxalate'

# Perform division and assign the result to the specified range of columns
PILS_11_subset2.loc[:, start_col2:stop_col2] = PILS_11_subset2.loc[:, start_col2:stop_col2].div(PILS_11_subset1, axis=0)
PILS_11_subset2.mean()

Time_Start    2022-06-11 15:27:10
Time_Stop     2022-06-11 15:32:10
Time_Mid      2022-06-11 15:29:40
Sodium                   0.223615
Potassium                0.015436
Magnesium                0.030409
Calcium                  0.067789
Chloride                 0.189828
Nitrate                  0.226372
Sulfate                  0.241657
Oxalate                  0.004895
dtype: object

In [629]:
PILS_11_subset2['Nso4'] = PILS_11_subset2['Sulfate'] - (0.253 * PILS_11_subset2['Sodium'])
PILS_11_subset2['ratio'] = PILS_11_subset2['Chloride'] / PILS_11_subset2['Sodium']
PILS_11_subset2.mean()

Time_Start    2022-06-11 15:27:10
Time_Stop     2022-06-11 15:32:10
Time_Mid      2022-06-11 15:29:40
Sodium                   0.223615
Potassium                0.015436
Magnesium                0.030409
Calcium                  0.067789
Chloride                 0.189828
Nitrate                  0.226372
Sulfate                  0.241657
Oxalate                  0.004895
Nso4                     0.185083
ratio                    0.848904
dtype: object

In [630]:
PILS_11_subset1.mean()

3.26454

In [631]:
# Specify the target timestamps
start_time = pd.to_datetime("2022-06-11 20:08:00")
stop_time = pd.to_datetime("2022-06-11 20:16:00")

# Find the nearest indices to the target timestamps
start_index = np.argmin(np.abs(ds_2020["Time_Start"] - start_time))
stop_index = np.argmin(np.abs(ds_2020["Time_Start"] - stop_time))

# Extract the subset of data
PILS_11b_subset = ds_2020.iloc[start_index:stop_index + 1]  # Add 1 to include the stop_index
PILS_11b_subset2 = PILS_11b_subset.drop('Ammonium', axis=1)
columns_to_sum = ['Sodium', 'Potassium', 'Magnesium', 'Calcium', 'Chloride', 'Nitrate', 'Sulfate', 'Oxalate']

# Sum the selected columns and store the result in Africa_total1
PILS_11b_subset1 = PILS_11b_subset2[columns_to_sum].sum(axis = 1)
# Calculate the sum of values for each row (axis=1)

# Calculate the sum of all the values in Africa_total1
#Africa_total1 = Africa_total1.sum()

# Specify the range of columns for division
start_col2 = 'Sodium'
stop_col2 = 'Oxalate'

# Perform division and assign the result to the specified range of columns
PILS_11b_subset2.loc[:, start_col2:stop_col2] = PILS_11b_subset2.loc[:, start_col2:stop_col2].div(PILS_11b_subset1, axis=0)

PILS_11b_subset2.mean()

Time_Start    2022-06-11 20:11:30
Time_Stop     2022-06-11 20:16:30
Time_Mid      2022-06-11 20:14:00
Sodium                    0.09141
Potassium                 0.02414
Magnesium                0.057263
Calcium                  0.070693
Chloride                 0.134153
Nitrate                  0.291085
Sulfate                  0.277312
Oxalate                  0.053944
dtype: object

In [632]:
PILS_11b_subset2['Nso4'] = PILS_11b_subset2['Sulfate'] - (0.253 * PILS_11b_subset2['Sodium'])
PILS_11b_subset2['ratio'] = PILS_11b_subset2['Chloride'] / PILS_11b_subset2['Sodium']
PILS_11b_subset2.mean()

Time_Start    2022-06-11 20:11:30
Time_Stop     2022-06-11 20:16:30
Time_Mid      2022-06-11 20:14:00
Sodium                    0.09141
Potassium                 0.02414
Magnesium                0.057263
Calcium                  0.070693
Chloride                 0.134153
Nitrate                  0.291085
Sulfate                  0.277312
Oxalate                  0.053944
Nso4                     0.254185
ratio                    2.178497
dtype: object

In [633]:
PILS_11b_subset1.mean()

5.6376599999999994

In [634]:
# Specify the target timestamps
start_time = pd.to_datetime("2022-06-13 14:30:00")
stop_time = pd.to_datetime("2022-06-13 14:46:00")

# Find the nearest indices to the target timestamps
start_index = np.argmin(np.abs(ds_2020["Time_Start"] - start_time))
stop_index = np.argmin(np.abs(ds_2020["Time_Start"] - stop_time))

# Extract the subset of data
PILS_13_subset = ds_2020.iloc[start_index:stop_index + 1]  # Add 1 to include the stop_index
PILS_13_subset2 = PILS_13_subset.drop('Ammonium', axis=1)
columns_to_sum = ['Sodium', 'Potassium', 'Magnesium', 'Calcium', 'Chloride', 'Nitrate', 'Sulfate', 'Oxalate']

# Sum the selected columns and store the result in Africa_total1
PILS_13_subset1 = PILS_13_subset2[columns_to_sum].sum(axis = 1)
# Calculate the sum of values for each row (axis=1)

# Calculate the sum of all the values in Africa_total1
#Africa_total1 = Africa_total1.sum()

# Specify the range of columns for division
start_col2 = 'Sodium'
stop_col2 = 'Oxalate'

# Perform division and assign the result to the specified range of columns
PILS_13_subset2.loc[:, start_col2:stop_col2] = PILS_13_subset2.loc[:, start_col2:stop_col2].div(PILS_13_subset1, axis=0)
PILS_13_subset2.mean()

Time_Start    2022-06-13 14:38:06.999999744
Time_Stop     2022-06-13 14:43:01.666666496
Time_Mid      2022-06-13 14:40:34.333333504
Sodium                             0.201648
Potassium                          0.009431
Magnesium                           0.02587
Calcium                            0.021962
Chloride                           0.291789
Nitrate                            0.175438
Sulfate                            0.268359
Oxalate                            0.005503
dtype: object

In [635]:
PILS_13_subset2['Nso4'] = PILS_13_subset2['Sulfate'] - (0.253 * PILS_13_subset2['Sodium'])
PILS_13_subset2['ratio'] = PILS_13_subset2['Chloride'] / PILS_13_subset2['Sodium']
PILS_13_subset2.mean()

Time_Start    2022-06-13 14:38:06.999999744
Time_Stop     2022-06-13 14:43:01.666666496
Time_Mid      2022-06-13 14:40:34.333333504
Sodium                             0.201648
Potassium                          0.009431
Magnesium                           0.02587
Calcium                            0.021962
Chloride                           0.291789
Nitrate                            0.175438
Sulfate                            0.268359
Oxalate                            0.005503
Nso4                               0.217342
ratio                              1.452314
dtype: object

In [636]:
PILS_13_subset1.mean()

5.30466

In [637]:
# Specify the target timestamps
start_time = pd.to_datetime("2022-06-13 19:46:00")
stop_time = pd.to_datetime("2022-06-13 20:09:00")

# Find the nearest indices to the target timestamps
start_index = np.argmin(np.abs(ds_2020["Time_Start"] - start_time))
stop_index = np.argmin(np.abs(ds_2020["Time_Start"] - stop_time))

# Extract the subset of data
PILS_13b_subset = ds_2020.iloc[start_index:stop_index + 1]  # Add 1 to include the stop_index
PILS_13b_subset2 = PILS_13b_subset.drop('Ammonium', axis=1)
columns_to_sum = ['Sodium', 'Potassium', 'Magnesium', 'Calcium', 'Chloride', 'Nitrate', 'Sulfate', 'Oxalate']

# Sum the selected columns and store the result in Africa_total1
PILS_13b_subset1 = PILS_13b_subset2[columns_to_sum].sum(axis = 1)
# Calculate the sum of values for each row (axis=1)

# Calculate the sum of all the values in Africa_total1
#Africa_total1 = Africa_total1.sum()

# Specify the range of columns for division
start_col2 = 'Sodium'
stop_col2 = 'Oxalate'

# Perform division and assign the result to the specified range of columns
PILS_13b_subset2.loc[:, start_col2:stop_col2] = PILS_13b_subset2.loc[:, start_col2:stop_col2].div(PILS_13b_subset1, axis=0)
PILS_13b_subset2.mean()

Time_Start    2022-06-13 19:47:13
Time_Stop     2022-06-13 19:52:13
Time_Mid      2022-06-13 19:49:43
Sodium                   0.084765
Potassium                 0.04707
Magnesium                0.036042
Calcium                  0.311758
Chloride                 0.127417
Nitrate                  0.155545
Sulfate                  0.232301
Oxalate                  0.005101
dtype: object

In [638]:
PILS_13b_subset2['Nso4'] = PILS_13b_subset2['Sulfate'] - (0.253 * PILS_13b_subset2['Sodium'])
PILS_13b_subset2['ratio'] = PILS_13b_subset2['Chloride'] / PILS_13b_subset2['Sodium']
PILS_13b_subset2.mean()

Time_Start    2022-06-13 19:47:13
Time_Stop     2022-06-13 19:52:13
Time_Mid      2022-06-13 19:49:43
Sodium                   0.084765
Potassium                 0.04707
Magnesium                0.036042
Calcium                  0.311758
Chloride                 0.127417
Nitrate                  0.155545
Sulfate                  0.232301
Oxalate                  0.005101
Nso4                     0.210855
ratio                    1.503187
dtype: object

In [639]:
PILS_13b_subset1.mean()

3.14635

In [640]:
# Specify the target timestamps
start_time = pd.to_datetime("2022-06-14 16:05:00")
stop_time = pd.to_datetime("2022-06-14 16:25:00")

# Find the nearest indices to the target timestamps
start_index = np.argmin(np.abs(ds_2020["Time_Start"] - start_time))
stop_index = np.argmin(np.abs(ds_2020["Time_Start"] - stop_time))

# Extract the subset of data
PILS_14_subset = ds_2020.iloc[start_index:stop_index + 1]  # Add 1 to include the stop_index
PILS_14_subset2 = PILS_14_subset.drop('Ammonium', axis=1)
columns_to_sum = ['Sodium', 'Potassium', 'Magnesium', 'Calcium', 'Chloride', 'Nitrate', 'Sulfate', 'Oxalate']

# Sum the selected columns and store the result in Africa_total1
PILS_14_subset1 = PILS_14_subset2[columns_to_sum].sum(axis = 1)
# Calculate the sum of values for each row (axis=1)

# Calculate the sum of all the values in Africa_total1
#Africa_total1 = Africa_total1.sum()

# Specify the range of columns for division
start_col2 = 'Sodium'
stop_col2 = 'Oxalate'

# Perform division and assign the result to the specified range of columns
PILS_14_subset2.loc[:, start_col2:stop_col2] = PILS_14_subset2.loc[:, start_col2:stop_col2].div(PILS_14_subset1, axis=0)
PILS_14_subset2.mean()

Time_Start    2022-06-14 16:02:11
Time_Stop     2022-06-14 16:07:11
Time_Mid      2022-06-14 16:04:41
Sodium                   0.154587
Potassium                0.007378
Magnesium                0.022808
Calcium                  0.030114
Chloride                 0.199888
Nitrate                  0.271856
Sulfate                  0.309952
Oxalate                  0.003416
dtype: object

In [641]:
PILS_14_subset2['Nso4'] = PILS_14_subset2['Sulfate'] - (0.253 * PILS_14_subset2['Sodium'])
PILS_14_subset2['ratio'] = PILS_14_subset2['Chloride'] / PILS_14_subset2['Sodium']
PILS_14_subset2.mean()

Time_Start    2022-06-14 16:02:11
Time_Stop     2022-06-14 16:07:11
Time_Mid      2022-06-14 16:04:41
Sodium                   0.154587
Potassium                0.007378
Magnesium                0.022808
Calcium                  0.030114
Chloride                 0.199888
Nitrate                  0.271856
Sulfate                  0.309952
Oxalate                  0.003416
Nso4                     0.270842
ratio                    1.293043
dtype: object

In [642]:
PILS_14_subset1.mean()

3.85866

In [643]:
# Specify the target timestamps
start_time = pd.to_datetime("2022-06-17 16:20:00")
stop_time = pd.to_datetime("2022-06-17 16:47:00")

# Find the nearest indices to the target timestamps
start_index = np.argmin(np.abs(ds_2020["Time_Start"] - start_time))
stop_index = np.argmin(np.abs(ds_2020["Time_Start"] - stop_time))

# Extract the subset of data
PILS_17_subset = ds_2020.iloc[start_index:stop_index + 1]  # Add 1 to include the stop_index
PILS_17_subset2 = PILS_17_subset.drop('Ammonium', axis=1)
columns_to_sum = ['Sodium', 'Potassium', 'Magnesium', 'Calcium', 'Chloride', 'Nitrate', 'Sulfate', 'Oxalate']

# Sum the selected columns and store the result in Africa_total1
PILS_17_subset1 = PILS_17_subset2[columns_to_sum].sum(axis = 1)
# Calculate the sum of values for each row (axis=1)

# Calculate the sum of all the values in Africa_total1
#Africa_total1 = Africa_total1.sum()

# Specify the range of columns for division
start_col2 = 'Sodium'
stop_col2 = 'Oxalate'

# Perform division and assign the result to the specified range of columns
PILS_17_subset2.loc[:, start_col2:stop_col2] = PILS_17_subset2.loc[:, start_col2:stop_col2].div(PILS_17_subset1, axis=0)
PILS_17_subset2.mean()

Time_Start    2022-06-17 16:23:09.500000
Time_Stop     2022-06-17 16:28:09.500000
Time_Mid      2022-06-17 16:25:39.500000
Sodium                          0.280222
Potassium                       0.008736
Magnesium                       0.025071
Calcium                         0.002052
Chloride                         0.43273
Nitrate                         0.117366
Sulfate                         0.132869
Oxalate                         0.000953
dtype: object

In [644]:
PILS_17_subset2['Nso4'] = PILS_17_subset2['Sulfate'] - (0.253 * PILS_17_subset2['Sodium'])
PILS_17_subset2['ratio'] = PILS_17_subset2['Chloride'] / PILS_17_subset2['Sodium']
PILS_17_subset2.mean()

Time_Start    2022-06-17 16:23:09.500000
Time_Stop     2022-06-17 16:28:09.500000
Time_Mid      2022-06-17 16:25:39.500000
Sodium                          0.280222
Potassium                       0.008736
Magnesium                       0.025071
Calcium                         0.002052
Chloride                         0.43273
Nitrate                         0.117366
Sulfate                         0.132869
Oxalate                         0.000953
Nso4                            0.061973
ratio                           1.542886
dtype: object

In [645]:
PILS_17_subset1.mean()

12.904860000000001

In [646]:

'''# Concatenate the dataframes vertically (along rows)
Africa_total = pd.concat([PILS_13_subset2, PILS_13b_subset2, PILS_14_subset2], axis=0)
Africa_total.reset_index(drop=True, inplace=True)
#Africa_total2 = Africa_total.drop('Ammonium', axis=1)
Africa_total.mean()

# Select the columns you want to sum
#columns_to_sum = ['Sodium', 'Potassium', 'Magnesium', 'Calcium', 'Chloride', 'Nitrate', 'Sulfate', 'Oxalate']

# Sum the selected columns and store the result in Africa_total1
#Africa_total1 = Africa_total2[columns_to_sum].sum(axis = 1)
# Calculate the sum of values for each row (axis=1)

# Calculate the sum of all the values in Africa_total1
#Africa_total1 = Africa_total1.sum()

#Africa_total1
# Specify the range of columns for division
#start_col2 = 'Sodium'
#stop_col2 = 'Oxalate'

# Perform division and assign the result to the specified range of columns
#Africa_total2.loc[:, start_col2:stop_col2] = Africa_total2.loc[:, start_col2:stop_col2].div(Africa_total1, axis=0)
#Africa_total2.mean()
#Africa_total1 = (Africa_total['Sodium'] + Africa_total['Potassium'] + Africa_total['Magnesium'] + Africa_total['Calcium'] +  Africa_total['Chloride'] + Africa_total['Nitrate'] + Africa_total['Sulfate'] + Africa_total['Oxalate'])
#Africa_total.sum()
# Reset the index of the DAY2b_subsetd dataframe'''



"# Concatenate the dataframes vertically (along rows)\nAfrica_total = pd.concat([PILS_13_subset2, PILS_13b_subset2, PILS_14_subset2], axis=0)\nAfrica_total.reset_index(drop=True, inplace=True)\n#Africa_total2 = Africa_total.drop('Ammonium', axis=1)\nAfrica_total.mean()\n\n# Select the columns you want to sum\n#columns_to_sum = ['Sodium', 'Potassium', 'Magnesium', 'Calcium', 'Chloride', 'Nitrate', 'Sulfate', 'Oxalate']\n\n# Sum the selected columns and store the result in Africa_total1\n#Africa_total1 = Africa_total2[columns_to_sum].sum(axis = 1)\n# Calculate the sum of values for each row (axis=1)\n\n# Calculate the sum of all the values in Africa_total1\n#Africa_total1 = Africa_total1.sum()\n\n#Africa_total1\n# Specify the range of columns for division\n#start_col2 = 'Sodium'\n#stop_col2 = 'Oxalate'\n\n# Perform division and assign the result to the specified range of columns\n#Africa_total2.loc[:, start_col2:stop_col2] = Africa_total2.loc[:, start_col2:stop_col2].div(Africa_total1, a

In [647]:
'''# Specify the range of columns for division
start_col2 = 'Sodium'
stop_col2 = 'Oxalate'

# Perform division and assign the result to the specified range of columns
Africa_total2.loc[:, start_col2:stop_col2] = Africa_total2.loc[:, start_col2:stop_col2].div(Africa_total1, axis=0)
Africa_total2.mean()'''

"# Specify the range of columns for division\nstart_col2 = 'Sodium'\nstop_col2 = 'Oxalate'\n\n# Perform division and assign the result to the specified range of columns\nAfrica_total2.loc[:, start_col2:stop_col2] = Africa_total2.loc[:, start_col2:stop_col2].div(Africa_total1, axis=0)\nAfrica_total2.mean()"

In [661]:

# Concatenate the dataframes vertically (along rows)
Ocean_total = pd.concat([PILS_2_subset2, PILS_22_subset2, PILS_2b_subset2], axis=0)
Ocean_total.reset_index(drop=True, inplace=True)
#Ocean_total2 = Ocean_total.drop('Ammonium', axis=1)

# Select the columns you want to sum
#columns_to_sum = ['Sodium', 'Potassium', 'Magnesium', 'Calcium', 'Chloride', 'Nitrate', 'Sulfate', 'Oxalate']

# Sum the selected columns and store the result in Africa_total1
#Ocean_total1 = Ocean_total2[columns_to_sum].sum(axis = 1)
# Calculate the sum of values for each row (axis=1)

# Calculate the sum of all the values in Africa_total1
#Africa_total1 = Africa_total1.sum()

# Specify the range of columns for division
#start_col2 = 'Sodium'
#stop_col2 = 'Oxalate'

# Perform division and assign the result to the specified range of columns
#Ocean_total2.loc[:, start_col2:stop_col2] = Ocean_total2.loc[:, start_col2:stop_col2].div(Ocean_total1, axis=0)
#Ocean_total2.mean()
#Africa_total1 = (Africa_total['Sodium'] + Africa_total['Potassium'] + Africa_total['Magnesium'] + Africa_total['Calcium'] +  Africa_total['Chloride'] + Africa_total['Nitrate'] + Africa_total['Sulfate'] + Africa_total['Oxalate'])
#Africa_total.sum()
# Reset the index of the DAY2b_subsetd dataframe
Ocean_total.mean()



Time_Start    2022-06-02 15:21:41.000000256
Time_Stop     2022-06-02 15:26:41.000000256
Time_Mid      2022-06-02 15:24:10.999999744
Sodium                              0.26675
Potassium                          0.015909
Magnesium                          0.026912
Calcium                            0.026197
Chloride                           0.139574
Nitrate                            0.204154
Sulfate                            0.316095
Oxalate                            0.004408
Nso4                               0.248607
ratio                              0.704101
dtype: object

In [660]:

# Concatenate the dataframes vertically (along rows)
Caribbean_total = pd.concat([PILS_10_subset2, PILS_10b_subset2, PILS_10c_subset2, PILS_11_subset2, PILS_11b_subset2, PILS_13_subset2, PILS_13b_subset2, PILS_14_subset2], axis=0)
Caribbean_total.reset_index(drop=True, inplace=True)
#Caribbean_total2 = Caribbean_total.drop('Ammonium', axis=1)

# Select the columns you want to sum
#columns_to_sum = ['Sodium', 'Potassium', 'Magnesium', 'Calcium', 'Chloride', 'Nitrate', 'Sulfate', 'Oxalate']

# Sum the selected columns and store the result in Africa_total1
#Caribbean_total1 = Caribbean_total[columns_to_sum].sum(axis = 1)
# Calculate the sum of values for each row (axis=1)

# Calculate the sum of all the values in Africa_total1
#Africa_total1 = Africa_total1.sum()

# Specify the range of columns for division
#start_col2 = 'Sodium'
#stop_col2 = 'Oxalate'

# Perform division and assign the result to the specified range of columns
#Caribbean_total2.loc[:, start_col2:stop_col2] = Caribbean_total2.loc[:, start_col2:stop_col2].div(Caribbean_total1, axis=0)
#Caribbean_total2.mean()
#Africa_total1 = (Africa_total['Sodium'] + Africa_total['Potassium'] + Africa_total['Magnesium'] + Africa_total['Calcium'] +  Africa_total['Chloride'] + Africa_total['Nitrate'] + Africa_total['Sulfate'] + Africa_total['Oxalate'])
#Africa_total.sum()
# Reset the index of the DAY2b_subsetd dataframe
Caribbean_total.mean()



Time_Start    2022-06-12 04:49:48.615384832
Time_Stop     2022-06-12 04:54:47.384615424
Time_Mid      2022-06-12 04:52:18.000000256
Sodium                             0.201263
Potassium                          0.014966
Magnesium                          0.031492
Calcium                            0.054696
Chloride                           0.229872
Nitrate                            0.198229
Sulfate                            0.257419
Oxalate                            0.012064
Nso4                               0.206499
ratio                              1.402932
dtype: object

In [650]:

# Concatenate the dataframes vertically (along rows)
North_America_total = pd.concat([PILS_7_subset2, PILS_7b_subset2, PILS_8_subset2, PILS_17_subset2], axis=0)
North_America_total.reset_index(drop=True, inplace=True)
#North_America_total2 = North_America_total.drop('Ammonium', axis=1)

# Select the columns you want to sum
#columns_to_sum = ['Sodium', 'Potassium', 'Magnesium', 'Calcium', 'Chloride', 'Nitrate', 'Sulfate', 'Oxalate']

# Sum the selected columns and store the result in Africa_total1
#North_America_total1 = North_America_total2[columns_to_sum].sum(axis = 1)
# Calculate the sum of values for each row (axis=1)

# Calculate the sum of all the values in Africa_total1
#Africa_total1 = Africa_total1.sum()

# Specify the range of columns for division
#start_col2 = 'Sodium'
#stop_col2 = 'Oxalate'

# Perform division and assign the result to the specified range of columns
#North_America_total2.loc[:, start_col2:stop_col2] = North_America_total2.loc[:, start_col2:stop_col2].div(North_America_total1, axis=0)
#North_America_total2.mean()
#Africa_total1 = (Africa_total['Sodium'] + Africa_total['Potassium'] + Africa_total['Magnesium'] + Africa_total['Calcium'] +  Africa_total['Chloride'] + Africa_total['Nitrate'] + Africa_total['Sulfate'] + Africa_total['Oxalate'])
#Africa_total.sum()
# Reset the index of the DAY2b_subsetd dataframe

North_America_total.mean()

Time_Start    2022-06-10 20:52:24.714285824
Time_Stop     2022-06-10 20:57:24.714285824
Time_Mid      2022-06-10 20:54:54.714285568
Sodium                             0.333473
Potassium                          0.007364
Magnesium                          0.027295
Calcium                            0.009965
Chloride                           0.432944
Nitrate                             0.07531
Sulfate                            0.112462
Oxalate                            0.001187
Nso4                               0.047501
ratio                               1.38741
dtype: object